# **Usage: Click "Run all" then scroll to plots at bottom of page**

In [7]:
#@title Imports
#%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
import ipywidgets as widgets
from IPython.display import display
# load experimental data stored in this github
!wget https://raw.githubusercontent.com/adrozman/ULI-noise-viz/main/ULI_selected_data.npz

--2025-10-29 19:39:27--  https://raw.githubusercontent.com/adrozman/ULI-noise-viz/main/ULI_selected_data.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99311441 (95M) [application/octet-stream]
Saving to: ‘ULI_selected_data.npz.1’

ULI_selected_data.n 100%[===================>]  94.71M   444MB/s    in 0.2s    

2025-10-29 19:39:28 (444 MB/s) - ‘ULI_selected_data.npz.1’ saved [99311441/99311441]



In [8]:
#@title Load Data
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "DejaVu Serif"

data=np.load("ULI_selected_data.npz",allow_pickle=True)
f= data['f']
spectra_mic_locs = data['spectra_mic_locs']
xyz_all = data['xyz']
noise = data['noise'].item()

all_conditions = list(noise.keys())
u_options = sorted(list(set([k[0] for k in all_conditions])))
yaw_options = sorted(list(set([k[1] for k in all_conditions])))
del data

In [9]:
#@title Widget creation and logic
mic_options_list = [(f"[{coords[0]:.2f}, {coords[1]:.2f}]", i)
    for i, coords in zip(range(7), spectra_mic_locs[0, :7, :2])]

u_widget = widgets.Dropdown(options=u_options,description='U (m/s):',value=10)
yaw_widget = widgets.Dropdown(options=yaw_options,description='Yaw (deg):',value=80)
# RPM widget is created with the *initial* list
rpm_widget = widgets.Dropdown(options=noise[(10,0)]['rpms'],description='RPM:',value=3000)
fq_widget = widgets.Dropdown(options=["BPF", "BROADBAND"],description='Contour Map Value:')
mic_widget = widgets.Dropdown(options=mic_options_list,description='Target Mic:')
spl_range_slider = widgets.FloatRangeSlider(value=[20, 80],min=0,max=120,step=5,
    description='SPL range',continuous_update=True,readout_format='.0f'
)

# u and rpm options need to update every time you change yaw.
def on_yaw_change(change):
    current_yaw = yaw_widget.value
    current_u = u_widget.value
    current_rpm = rpm_widget.value
    available_u = sorted({u for u, y in all_conditions if y == current_yaw})

    u_widget.unobserve(on_u_change, names='value')
    rpm_widget.unobserve(update_plot, names='value')
    fq_widget.unobserve(update_plot, names='value')
    mic_widget.unobserve(update_plot, names='value')
    spl_range_slider.unobserve(update_plot, names='value')

    u_widget.options = available_u
    if current_u in available_u:
        u_widget.value = current_u

    available_rpm = noise[(current_u, current_yaw)]['rpms']
    rpm_widget.options = available_rpm
    if current_rpm in available_rpm:
        rpm_widget.value = current_rpm
    else:
        rpm_widget.value = available_rpm[-2]

    u_widget.observe(on_u_change, names='value')
    rpm_widget.observe(update_plot, names='value')
    fq_widget.observe(update_plot, names='value')
    mic_widget.observe(update_plot, names='value')
    spl_range_slider.observe(update_plot, names='value')

    update_plot(None)

# yaw and rpm options need to update every time you change u.
def on_u_change(change):
    current_u = u_widget.value
    current_yaw = yaw_widget.value
    current_rpm = rpm_widget.value
    available_yaw = sorted({y for u, y in all_conditions if u == current_u})

    yaw_widget.unobserve(on_yaw_change, names='value')
    rpm_widget.unobserve(update_plot, names='value')
    fq_widget.unobserve(update_plot, names='value')
    mic_widget.unobserve(update_plot, names='value')
    spl_range_slider.unobserve(update_plot, names='value')

    yaw_widget.options = available_yaw
    if current_yaw in available_yaw:
        yaw_widget.value = current_yaw

    available_rpm = noise[(current_u, current_yaw)]['rpms']
    rpm_widget.options = available_rpm
    if current_rpm in available_rpm:
        rpm_widget.value = current_rpm
    else:
        rpm_widget.value = available_rpm[-2]

    yaw_widget.observe(on_yaw_change, names='value')
    rpm_widget.observe(update_plot, names='value')
    fq_widget.observe(update_plot, names='value')
    mic_widget.observe(update_plot, names='value')
    spl_range_slider.observe(update_plot, names='value')

    update_plot(None)

u_widget.observe(on_u_change, names='value')
yaw_widget.observe(on_yaw_change, names='value')

# the output will be the figure.
out_plot1 = widgets.Output(layout=widgets.Layout(height='300px'))

In [10]:
#@title Plotting Function
def plot_figs(u=u_widget, yaw=yaw_widget, rpm=rpm_widget, target_fq=fq_widget, target_mic=mic_widget, spl_limits=spl_range_slider):

    BPF = rpm / 60 * 5
    titles = ['BACKGROUND', 'TOTAL', 'TONAL']
    colors = ['gray', 'k', 'b']
    linewidths = [2.3, 2.3, 2.3]
    linestyles = ['dashed', 'solid', 'dashdot']

    rpm_ix = np.argmin(abs(np.array(noise[(u,yaw)]['rpms']) - rpm))

    bg = noise[(u,yaw)]['spectra']['background'][target_mic]
    tonal = noise[(u,yaw)]['spectra']['tonal'][rpm_ix, target_mic]
    total = noise[(u,yaw)]['spectra']['total'][rpm_ix, target_mic]

    if u == 0:
        xyz=xyz_all[0]
        spectra_mic_loc = spectra_mic_locs[0,target_mic]
    elif u==10:
        xyz=xyz_all[1]
        spectra_mic_loc = spectra_mic_locs[1,target_mic]
    else:
        xyz=xyz_all[2]
        spectra_mic_loc = spectra_mic_locs[2,target_mic]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4),gridspec_kw={'width_ratios': [3, 4]})

    # Spectra plot
    for i, SPL in enumerate((bg, total, tonal)):
        ax1.semilogx(f / BPF, SPL, label=titles[i], linewidth=linewidths[i], color=colors[i], linestyle=linestyles[i])

    # ax1.set_title(fr"{rpm} RPM, {u} m/s, {yaw}$\degree$ yaw at mic [{spectra_mic_loc[0]:.2f}, {spectra_mic_loc[1]:.2f}]", fontsize=15)
    ax1.set_xticks(np.arange(11))
    ax1.set_ylim(spl_limits[0], spl_limits[1])
    ax1.set_xlim(0.9, 40)
    ax1.set_xlabel("BPF Harmonic", fontsize=18)
    ax1.set_ylabel("SPL (dB)", fontsize=18)
    ax1.grid()
    ax1.tick_params(axis='both', which='both', direction='inout', labelsize=16, length=5, width=1.6)
    ax1.legend(fontsize=10, loc='upper right', ncols=3)

    # Directivity Plot
    X, Y = np.meshgrid(np.linspace(-2, 2, 51), np.linspace(-1, 1, 26))
    levels = np.linspace(spl_limits[0], spl_limits[1], 75)
    SPL = noise[(u,yaw)]['maps']['BPF'][rpm_ix] if target_fq=="BPF" else noise[(u,yaw)]['maps']['BROADBAND'][rpm_ix]
    Z = griddata((xyz[:, 0], xyz[:, 1]), SPL, (X, Y), method='linear')

    # fig2, ax2 = plt.subplots(layout='constrained', figsize=(4, 3))
    CS = ax2.contourf(X, Y, Z, levels, extend='both', cmap="jet")
    title = fr"1st BPF Directivity: {rpm} RPM, {u} m/s, {yaw}$\degree$ yaw"

    # Call the helper function
    # format_map(fig, ax2, CS, levels, title="")
    ax2.set_xlabel('X (m)', fontsize=14)
    ax2.set_ylabel('Y (m)', fontsize=14)
    ax2.tick_params(axis='both', labelsize=10)
    ax2.axis('equal')
    ax2.tick_params(axis='both', which='both', direction='inout', pad=7)
    ax2.tick_params(axis='both', which='major', labelsize=14, length=4, width=1.6)

    # make colorbar
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    ticks = np.linspace(levels[0], levels[-1], 10).tolist()
    cbar = fig.colorbar(CS, cax=cax, ticks=ticks)
    cbar.ax.set_ylabel("SPL (dB)", fontsize=14)
    cbar.ax.set_yticklabels([f'{x:.2f}' for x in ticks])

    ax2.set_ylim((-1., 1.))
    # plt.show(fig2)

    fig.canvas.draw()
    ax2_pos = ax2.get_position()
    ax1_pos = ax1.get_position()

    ax1.set_position([
        ax1_pos.x0,         # Keep original left
        ax2_pos.y0,         # Match ax2's bottom
        ax1_pos.width,      # Keep original width
        ax2_pos.height      # Match ax2's height
    ])

    out_plot1.clear_output(wait=True)
    with out_plot1:
        plt.show(fig)
        plt.close(fig)

def update_plot(change):
    plot_figs(
        u=u_widget.value,
        yaw=yaw_widget.value,
        rpm=rpm_widget.value,
        target_fq=fq_widget.value,
        target_mic=mic_widget.value,
        spl_limits=spl_range_slider.value
    )
# remaining widgets update plot every time they're changed - no other changes necessary.
rpm_widget.observe(update_plot, names='value')
fq_widget.observe(update_plot, names='value')
mic_widget.observe(update_plot, names='value')
spl_range_slider.observe(update_plot, names='value')

In [11]:
#@title Interactive Plot
controls = widgets.VBox([
    widgets.HBox([u_widget, yaw_widget, rpm_widget]),
    widgets.HBox([fq_widget, mic_widget, spl_range_slider]),
])

display(controls, out_plot1)

update_plot(None) # initialize

Output(layout=Layout(height='300px'))